transferring columns from [master TML spreadsheet](https://wsponline.sharepoint.com/:x:/r/sites/WDS-ClimateResilienceandSustainabilityCanada/_layouts/15/Doc.aspx?sourcedoc=%7B0A099FC0-DDE6-4789-9A0B-3645CB9D93E4%7D&file=Master%20TML%20Listing%20Updated%202025%20-%20Climate%20Adaptation.xlsx&action=default&mobileredirect=true) and adding to [poles.csv](https://wsponline.sharepoint.com/:x:/r/sites/WDS-ClimateResilienceandSustainabilityCanada/Shared%20Documents/General/Projects/CA0060027.8940%20-%20NF%20Power%20CRiVA/Client%20Data/Extracted%20CSVs/Coastline%20v2%20with%20elevation%20and%20region/Poles.csv?d=w6109d5e6a3fa4d19bc788353664802c9&csf=1&web=1&e=rSMXuj)

column AV is Transmissi, contains line number column C in poles

In [44]:
import pandas as pd
import numpy as np

In [ ]:
tmlf = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Poles-tr\Master TML Listing Updated 2025 - Climate Adaptation.xlsx"
tml = pd.read_excel(tmlf, header = 3)
polesf = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Coastline CSVs\Exports\v2 no interp with elev and reg\Poles.csv"
poles = pd.read_csv(polesf)

C:\Users\CAMG038492\AppData\Local\Temp\ipykernel_45432\4047890052.py:4: DtypeWarning: Columns (11,12,15,21,22,34,37,39,43,46,47,48,50,51,53,54,55,67,69) have mixed types. Specify dtype option on import or set low_memory=False.
  poles = pd.read_csv(polesf)


In [46]:
filter = True
if filter:
    poles = poles.dropna(subset = 'Transmissi').copy(deep = True)
tml = tml.rename(columns = {"VOLTAGE\nkV": "VOLTAGE kV", "YEAR \nBUILT": "YEAR BUILT"})

In [47]:
poles.Transmissi.value_counts()

Transmissi
65L-1      4
124L-88    3
57L-45     3
416L-94    3
416L-85    3
          ..
51L-89     1
51L-110    1
73L-73     1
13L-48     1
24L-169    1
Name: count, Length: 7470, dtype: int64

In [48]:
def v_findline(row):
    string = row['Transmissi']
    if isinstance(string,str):
        pre = int(string.split("L")[0])
    elif np.isnan(string):
        pre = string
    else:
        pre = string
        print(f"{pre} unchanged")
    return pre

In [49]:
def v_findsub(row):
    string = row['Transmissi']
    if isinstance(string,str):
        pre = string.split("-")[1]
        try: pre = int(pre)
        except:
            try:
                number = int(''.join([char for char in pre if char.isdigit()]))
                pre = number
            except:
                pre = ""
    elif np.isnan(string):
        pre = ""
    else:
        pre = string
        print(f"{pre} unchanged")
    return pre

In [50]:
poles['Transmissi-pre'] = poles.apply(v_findline, axis = 1)
poles['Transmissi-sub'] = poles.apply(v_findsub, axis = 1)

In [51]:
poles[['Transmissi','Transmissi-pre','Transmissi-sub']].dropna()

,Transmissi,Transmissi-pre,Transmissi-sub
129187,12L-33,12,33
289127,57L-32,57,32
289128,57L-33,57,33
289129,57L-34,57,34
289130,57L-35,57,35
...,...,...,...
369886,51L-89,51,89
369888,51L-110,51,110
374594,73L-73,73,73
375276,13L-48,13,48


In [52]:
# using poles[transmissi] columns, add tml 'CONDUCTOR' 'STRUCTURES' 'LOOP/RADIAL' 'VOLTAGE kV' 'YEAR BUILT' to poles
example = 3
tml[tml['LINE #']==3]

,AREA,REGION,LINE #,LOCATION FROM,LOCATION TO,LENGTH\nKM,CONDUCTOR,STRUCTURES,LOOP/RADIAL,VOLTAGE kV,YEAR BUILT,*REBUILD\nYEAR,Age
0,1,STJ,3,Petty Harbour (1),Point A (str 13),0.9,4/0 AASC STAND,S POLE (STEEL),Radial,33,2002,2060,23
1,1,STJ,3,Point A (str 13),Point B (str 61),3.4,4/0 AASC STAND,S POLE (STEEL),Radial,33,2004,2062,21
2,1,STJ,3,Point B (str 61),Goulds (82),1.3,4/0 AASC STAND,S POLE (WOOD),Radial,33,2004,2062,21


In [53]:
def v_fromnum(row, col):
    string = row[col]
    if not isinstance(string,str):
        print(type(string))
    else:
        extract = string#.split("(")[1]
        try:
            number = int(''.join([char for char in extract if char.isdigit()]))
        except:
            number = ""
    return number


tml['from'] = tml.apply(v_fromnum, col = 'LOCATION FROM', axis = 1)
tml['to'] = tml.apply(v_fromnum, col = 'LOCATION TO', axis = 1)

tml[['from','to']]

,from,to
0,1,13
1,13,61
2,61,82
3,1,47
4,47,191
...,...,...
320,44,47
321,1,94
322,95,109
323,25,55


In [54]:
def v_findinfo(row):
    line = row['Transmissi-pre']
    des = {'CONDUCTOR': "", 'STRUCTURES': "", 'LOOP/RADIAL': "", 'VOLTAGE kV': "", 'YEAR BUILT': ""}
    if np.isnan(line):
        return des
    try:
        line = int(row['Transmissi-pre'])
        section = row['Transmissi-sub']
        mat = tml[tml['LINE #'] == line]
        firstrow = mat.iloc[0]
        for key in des.keys():
            des[key] = firstrow[key]
    except ValueError:
        print(f"{row['Transmission-pre']} invalid")
    return des

testrow = poles.dropna(subset = ['Transmissi']).iloc[0]
v_findinfo(testrow)

{'CONDUCTOR': '350 CBLE NONSTD',
 'STRUCTURES': 'U/G CABLE',
 'LOOP/RADIAL': 'Loop',
 'VOLTAGE kV': np.int64(66),
 'YEAR BUILT': np.int64(1966)}

In [55]:
poles['attempt'] = poles.apply(v_findinfo, axis=1)
poles[["CONDUCTOR", "STRUCTURES", "LOOP/RADIAL", "VOLTAGE kV", "YEAR BUILT"]] = pd.DataFrame(poles.attempt.tolist(), index=poles.index)
poles = poles.drop(['attempt','Transmissi-pre','Transmissi-sub'], axis = 1)


In [56]:
poles.CONDUCTOR.value_counts()

CONDUCTOR
397.5 ACSR STAND     2348
477 ASC STAND        1916
4/0 AASC STAND       1582
715.5 ASC STAND       940
599.5 AASC STAND      760
559.5 AASC STAND      708
266.8 ACSR STAND      451
477 ASC               353
244.4 ACSR NONSTD     333
397.5 ASCR STAND      205
1000 CBLE NONSTD       61
350 CBLE NONSTD        60
477 ASC 'COSMOS'       33
Name: count, dtype: int64

In [57]:
poles

,Unnamed: 0,X,Y,Z,fid,OBJECTID,CreationUs,DateCreate,LastUser,DateModifi,...,Region.1,Nearest coast (km),"Nearest coast (lat,lon)",Nearest coast (lat),Nearest coast (lon),CONDUCTOR,STRUCTURES,LOOP/RADIAL,VOLTAGE kV,YEAR BUILT
129187,129187,-52.714133,47.576478,31,129188,457442,System,2018-05-14,System,2022-09-02,...,St. John's,1.740614,"[47.56956603934127, -52.69337526895343]",47.569566,-52.693375,350 CBLE NONSTD,U/G CABLE,Loop,66,1966
289127,289127,-53.312685,47.586316,32,289128,850696,tdbutt,2018-08-16,System,2022-06-27,...,Avalon,2.620301,"[47.58280062804658, -53.27824089899826]",47.582801,-53.278241,559.5 AASC STAND,H FRAME,Loop,66,2017
289128,289128,-53.314014,47.587631,38,289129,850697,tdbutt,2018-08-16,System,2022-06-27,...,Avalon,2.743981,"[47.58280062804658, -53.27824089899826]",47.582801,-53.278241,559.5 AASC STAND,H FRAME,Loop,66,2017
289129,289129,-53.315179,47.588881,41,289130,850698,tdbutt,2018-08-16,System,2022-06-27,...,Avalon,2.441966,"[47.5990829247372, -53.28642507225925]",47.599083,-53.286425,559.5 AASC STAND,H FRAME,Loop,66,2017
289130,289130,-53.316580,47.590383,49,289131,850699,tdbutt,2018-08-16,System,2022-06-27,...,Avalon,2.465069,"[47.5990958866659, -53.28643977617469]",47.599096,-53.286440,559.5 AASC STAND,H FRAME,Loop,66,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369886,369886,-52.971580,47.495530,40,369887,7717138,System,2024-06-03,System,2024-07-04,...,St. John's,2.424300,"[47.51322533787909, -52.99038292263374]",47.513225,-52.990383,477 ASC STAND,S POLE(DBL CCT),Loop,66,1986
369888,369888,-52.986630,47.488310,58,369889,7717140,System,2024-06-03,System,2024-07-04,...,St. John's,2.848278,"[47.5042959450019, -53.01616879245574]",47.504296,-53.016169,477 ASC STAND,S POLE(DBL CCT),Loop,66,1986
374594,374594,-52.848597,47.503182,210,374595,8282898,bucrane,2025-04-30,bucrane,2025-05-01,...,St. John's,6.957755,"[47.54360288166771, -52.91912794709703]",47.543603,-52.919128,715.5 ASC STAND,S POLE,Loop,66,1975
375276,375276,-52.732172,47.555184,83,375277,8342367,bucrane,2025-06-20,bucrane,2025-06-20,...,St. John's,2.700274,"[47.56265059365001, -52.69802989995986]",47.562651,-52.698030,1000 CBLE NONSTD,U/G CABLE,Loop,66,1978


In [ ]:
if export:= False:
    import os
    name = "Poles-Tr.csv"
    folder = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Poles-tr"
    poles.to_csv(os.path.join(folder,name))
    print(f"{name} saved to {folder}")